## TCN
参考连接：https://blog.csdn.net/qq_34107425/article/details/105522916


# 卷积输出长度计算

卷积操作的输出长度取决于输入长度、卷积核大小、步幅（stride）、填充（padding）和扩张（dilation）等参数。下面是输出长度的计算公式和示例。

## 1. 标准卷积（没有扩张）

对于标准的 1D 卷积，输出长度的计算公式为：

$$
\text{Output Length} = \left\lfloor \frac{\text{Input Length} + 2 \times \text{Padding} - \text{Kernel Size}}{\text{Stride}} + 1 \right\rfloor
$$

- **Input Length**：输入数据的时间步长（或序列长度）。
- **Kernel Size**：卷积核的大小。
- **Padding**：填充量，添加到输入两端的零值。
- **Stride**：步幅，卷积核每次滑动的步长。
- **Output Length**：卷积操作后的输出长度。

## 2. 扩张卷积（Dilated Convolution）

对于扩张卷积，输出长度的计算公式为：

$$
\text{Output Length} = \left\lfloor \frac{\text{Input Length} + 2 \times \text{Padding} - (\text{Kernel Size} - 1) \times \text{Dilation} - 1}{\text{Stride}} + 1 \right\rfloor
$$

- **Dilation**：扩张率，卷积核中元素之间的间距。

### 3. 示例计算

#### 示例 1：标准卷积

- **输入长度**：10
- **卷积核大小**：3
- **填充**：1
- **步幅**：1

输出长度计算：

$$
\text{Output Length} = \left\lfloor \frac{10 + 2 \times 1 - 3}{1} + 1 \right\rfloor = \left\lfloor \frac{10 + 2 - 3}{1} + 1 \right\rfloor = \left\lfloor 9 + 1 \right\rfloor = 10
$$

因此，输出长度为 **10**。

#### 示例 2：扩张卷积

- **输入长度**：10
- **卷积核大小**：3
- **填充**：1
- **步幅**：1
- **扩张**：2

输出长度计算：

$$
\text{Output Length} = \left\lfloor \frac{10 + 2 \times 1 - (3 - 1) \times 2 - 1}{1} + 1 \right\rfloor = \left\lfloor \frac{10 + 2 - 4 - 1}{1} + 1 \right\rfloor = \left\lfloor 7 + 1 \right\rfloor = 8
$$

因此，输出长度为 **8**。

#### 示例 3：带步幅的卷积

- **输入长度**：10
- **卷积核大小**：3
- **填充**：1
- **步幅**：2

输出长度计算：

$$
\text{Output Length} = \left\lfloor \frac{10 + 2 \times 1 - 3}{2} + 1 \right\rfloor = \left\lfloor \frac{10 + 2 - 3}{2} + 1 \right\rfloor = \left\lfloor \frac{9}{2} + 1 \right\rfloor = \left\lfloor 4.5 + 1 \right\rfloor = 5
$$

因此，输出长度为 **5**。

## 4. 输出长度公式总结

一般化的输出长度公式是：

$$
\text{Output Length} = \left\lfloor \frac{\text{Input Length} + 2 \times \text{Padding} - (\text{Kernel Size} - 1) \times \text{Dilation} - 1}{\text{Stride}} + 1 \right\rfloor
$$

- **Padding**、**Dilation** 和 **Stride** 会影响输出长度的变化。
- 增加步幅会缩小输出长度，增加填充或扩张会使输出长度增加。

## 5. `Chomp1d` 的作用

- **`Chomp1d`** 模块用于修剪卷积后可能多出来的部分，确保输出长度与输入长度一致，特别是在使用扩张卷积时。
- 它通过去除卷积操作中多余的时间步，确保卷积后数据的时间步长与输入数据一致。



In [1]:
# 导入库
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


In [2]:
# 这个函数是用来修剪卷积之后的数据的尺寸，让其与输入数据尺寸相同。
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


In [3]:
# 这个就是TCN的基本模块，包含8个部分，两个（卷积+修剪+relu+dropout）
# 里面提到的downsample就是下采样，其实就是实现残差链接的部分。不理解的可以无视这个
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


In [ ]:
# TCN的主网络
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


In [8]:
from torchsummary import summary
import torch

# 检查是否有可用的 GPU，如果有则使用 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 假设的输入数据
batch_size = 4   # 批次大小
num_inputs = 3   # 输入通道数
sequence_length = 100  # 序列长度

# 创建一个形状为 (batch_size, num_inputs, sequence_length) 的假数据
x = torch.randn(batch_size, num_inputs, sequence_length).to(device)

# 定义 TCN 模型
num_channels = [16, 32, 64]  # 定义每一层的输出通道数
kernel_size = 3              # 卷积核大小
dropout = 0.2                # Dropout 比例

# 实例化 TemporalConvNet 模型
model = TemporalConvNet(num_inputs, num_channels, kernel_size, dropout).to(device)

# 打印模型的每一层信息和输出形状
summary(model, input_size=(num_inputs, sequence_length))

# 前向传播
output = model(x)

# 打印输出的形状
print("Output shape:", output.shape)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 16, 102]             160
            Conv1d-2              [-1, 16, 102]             160
           Chomp1d-3              [-1, 16, 100]               0
           Chomp1d-4              [-1, 16, 100]               0
              ReLU-5              [-1, 16, 100]               0
              ReLU-6              [-1, 16, 100]               0
           Dropout-7              [-1, 16, 100]               0
           Dropout-8              [-1, 16, 100]               0
            Conv1d-9              [-1, 16, 102]             784
           Conv1d-10              [-1, 16, 102]             784
          Chomp1d-11              [-1, 16, 100]               0
          Chomp1d-12              [-1, 16, 100]               0
             ReLU-13              [-1, 16, 100]               0
             ReLU-14              [-1, 